In [5]:
library("MASS")
library("Ecdat")

#### Question 5.1
In the first part we are calculating VaR and ES by using the historical return data of the S&P returns. Worth noting that VaR and ES values will be positive, even though they signify a loss to the investment. A negative VaR/ES number signifies a theoretical gain.

In [11]:
data("SP500")
alpha = 0.95
position_value = 1000000

In [12]:
# calculate VaR using quantile function
var = as.numeric(quantile(SP500$r500, probs = alpha))

# ES is the expected value of the tail, i.e. all values less than or equal to VaR
es = mean(SP500$r500[SP500$r500 >= var])

# convert from percent to dollars
dollar_var = var * position_value
dollar_es = es * position_value

sprintf('The dollar VaR of holding a $1m position in S&P500 over the given time period is %.2f and the ES is %.2f', dollar_var, dollar_es)

[1] "The dollar VaR of holding a $1m position in S&P500 over the given time period is 16263.13 and the ES is 22560.86"

#### Question 5.2
Now we fit the return data to a T distribution. After this, we use the t-distribution quantile to calculate VaR and ES.

In [13]:
fit = fitdistr(SP500$r500, densfun="t", lower=c(0, 0.001), start=list(m=0, s=sd(SP500$r500)**2, df = 4))

# extract the distribution parameters
mu = as.numeric(fit$estimate['m'])
sigma = as.numeric(fit$estimate['s'])
df = as.numeric(fit$estimate['df'])
sprintf('The parameters of the fitted t-distribution are: loc=%.6f, scale=%.6f, df=%.2f', mu, sigma, df)

# using the same formulas as Q4 to compute the analytic VaR/ES
# but in R form
quantile = qt(alpha, df=df)

var_t = mu + sigma * quantile
dollar_var_t = var_t * position_value

es_tilda = mu + dt(quantile, df=df) / (1 - alpha) * (df + quantile**2) / (df - 1)
es_t = mu + sigma * es_tilda
dollar_es_t = es_t * position_value

sprintf('The dollar VaR of holding a $1m position in S&P500 over the given time period is %.2f and the ES is %.2f', dollar_var_t, dollar_es_t)

[1] "The parameters of the fitted t-distribution are: loc=0.000047, scale=0.007290, df=4.00"

[1] "The dollar VaR of holding a $1m position in S&P500 over the given time period is 15587.51 and the ES is 23395.18"